# Cleaning statement of vote files

Copyright and distributed under an [MIT License](https://opensource.org/licenses/MIT).

In [1]:
# Data processing libraries
import pandas as pd
import numpy as np

pd.options.display.max_columns = 200
pd.options.display.max_rows = 100
idx = pd.IndexSlice

import json

## Background
Official precinct-level election results are available in the "Statement of Votes" (SOV) records from the [Boulder County Elections Division](https://www.bouldercounty.org/elections/by-year/) and [Colorado Secretary of State](https://www.coloradosos.gov/pubs/elections/Results/Archives.html) in Excel format.

Are the results of individual contests available at the level of precincts in an accessible format (XLS, CSV)?

| Year | Boulder | State |
| --- | --- | --- |
| 2022 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2022/12/2022G-Boulder-County-Official-Statement-of-Votes.xlsx) | ❌ |
| 2021 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2021/11/2021-Boulder-County-Coordinated-Election-Official-Statement-of-Votes-1.xlsx) | ❌ |
| 2020 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2020/11/2020-Boulder-County-General-Election-Official-Statement-of-Votes.xlsx) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2020/2020GEPrecinctLevelResultsPosted.xlsx) |
| 2019 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2019/11/2019C-Official-Statement-Of-Votes-SOV.xls) | ❌ |
| 2018 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2018/11/2018-General-Election-Official-Statement-Of-Votes.xlsx) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2018/2018GEPrecinctLevelResults.xlsx) |
| 2017 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/11/Results_SOV_Final.xlsx) | ❌ |
| 2016 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2016-general-election-results-final-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2016/General/2016GeneralTurnoutPrecinctLevel.xlsx) |
| 2015 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2015-election-sov.xls) | ❌ |
| 2014 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2014-general-election-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2014/2014GeneralPrecinctTurnout.xlsx) |
| 2013 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2013-election-sov.xls) | ❌ |
| 2012 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2012-general-election-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2012/2012GeneralPrecinctLevelTurnout.xlsx) |
| 2011 | [⚠️](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2011-election-sov.pdf) | ❌ |
| 2010 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/09/2010-general-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2010/general/2010GeneralPrecinctTurnout.xlsx) |
| 2009 | ❌ | ❌ |
| 2008 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/12/2008-general-election-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2008/2008GeneralPrecinctTurnout.xlsx) |
| 2007 | ❌ | ❌ |
| 2006 | ❌ | [✅](https://www.coloradosos.gov/pubs/elections/Results/2006/2006GeneralPrecinctBallotsCast.xlsx) | 
| 2005 | ❌ | ❌ |
| 2004 | ❌ | [✅](https://www.coloradosos.gov/pubs/elections/Results/2004/2004GeneralPrecinctBallotsCast.xlsx) |

Load up a list of precinct numbers within the City of Boulder.

In [2]:
with open('precincts.json','r') as f:
    precincts = json.load(f)

## Boulder County Clerk data

In [3]:
boco_original_path = './Original Data/Boulder County/'
boco_cleaned_path = './Cleaned Data/Boulder County/'

In [4]:
common_cols = ['Precinct Name','Active Voters','Total Ballots','Contest Name','Choice Name','Party','Total Votes']

In [5]:
party_map = {
    'ACN':'Constitution',
    'DEM':'Democratic',
    'GRN':'Green',
    'IND':'Independent',
    'LBR':'Libertarian',
    'REP':'Republican',
    'UAF':'Unaffiliated',
    'NP':'No Party',
    'UNI':'Unity',
    'APV':'Approval',
    'CTR':'Center'
}

In [6]:
choice_name_map = {
    'NO/AGAINST':'No',
    'YES/FOR':'Yes',
    'NO':'No',
    'YES':'Yes',
    'Against the Measure':'No',
    'For the Measure':'Yes',
    'AGAINST THE MEASURE':'No',
    'FOR THE MEASURE':'Yes',
    'No/Against':'No',
    'Yes/For':'Yes',
}

### 2022

In [7]:
# Read in file
sov22_raw_df = pd.read_excel(
    boco_original_path + '2022-general-sov.xlsx',
    dtype={'Precinct Name \n(Long)':str}
)

# Rename columns for consistency
sov22_raw_df = sov22_raw_df.rename(
    columns = {
        'Contest Title':'Contest Name',
        'Precinct Name \n(Short)':'Precinct Name (Short)',
        'Precinct Name \n(Long)':'Precinct Name',
        'Active \nVoters':'Active Voters',
        'Total \nBallots':'Total Ballots',
        'Total \nVotes':'Total Votes',
        'Total \nUndervotes':'Total Undervotes', 
        'Total \nOvervotes':'Total Overvotes'
    }
)

# Map party abbreviations to fuller name
sov22_raw_df['Party'] = sov22_raw_df['Party'].map(party_map)

# Subset to common columns
sov22_df = sov22_raw_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov22_df.replace({'Choice Name':choice_name_map},inplace=True)

# Write common columns to disk
sov22_df.to_csv(
    boco_cleaned_path + '2022-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov22_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,"2151907002, 2171907636",1398.0,1030,United States Senator,Michael Bennet,Democratic,664
1,"2151907002, 2171907636",1398.0,1030,United States Senator,Joe O'Dea,Republican,340


### 2021

In [8]:
sov21_df = pd.read_excel(
    boco_original_path + '2021-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov21_df = sov21_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

# Cast choices to consistent names
sov21_df.replace({'Choice Name':choice_name_map},inplace=True)

# Subset to common columns
sov21_df = sov21_df.reindex(columns=common_cols)

# Map party abbreviations to fuller name
sov21_df['Party'] = sov21_df['Party'].map(party_map)

# Write common columns to disk
sov21_df.to_csv(
    boco_cleaned_path + '2021-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov21_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2181007800,441,263,City of Boulder Council Candidates,Michael Christy,NaN,127
1,2181007800,441,263,City of Boulder Council Candidates,Mark Wallach,NaN,126


### 2020

In [9]:
sov20_df = pd.read_excel(
    boco_original_path + '2020-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov20_df = sov20_df.rename(
    columns = {
        'Active\nVoters':'Active Voters'
    }
)

# Subset to common columns
sov20_df = sov20_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov20_df.replace({'Choice Name':choice_name_map},inplace=True)

# Map party abbreviations to fuller name
sov20_df['Party'] = sov20_df['Party'].map(party_map)

# Write common columns to disk
sov20_df.to_csv(
    boco_cleaned_path + '2020-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov20_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2163307100,1547,1399,Presidential Electors,Joseph R. Biden / Kamala D. Harris,Democratic,997
1,2163307100,1547,1399,Presidential Electors,Donald J. Trump / Michael R. Pence,Republican,355


### 2019

In [10]:
sov19_df = pd.read_excel(
    boco_original_path + '2019-general-sov.xls',
    dtype={'Precinct Name':str}
)

sov19_df = sov19_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov19_df['Fraction'] = sov19_df['Total Votes']/sov19_df['Total Ballots']
sov19_df['Turnout'] = sov19_df['Total Votes']/sov19_df['Active Voters']

# Subset to common columns
sov19_df = sov19_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov19_df.replace({'Choice Name':choice_name_map},inplace=True)

# Map party abbreviations to fuller name
sov19_df['Party'] = sov19_df['Party'].map(party_map)

# Write common columns to disk
sov19_df.to_csv(
    boco_cleaned_path + '2019-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov19_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2181007800,428,266,City of Boulder Council Candidates,Brian Dolan,NaN,94
1,2181007800,428,266,City of Boulder Council Candidates,Rachel Friend,NaN,97


### 2018

In [11]:
sov18_df = pd.read_excel(
    boco_original_path + '2018-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov18_df = sov18_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

# Subset to common columns
sov18_df = sov18_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov18_df.replace({'Choice Name':choice_name_map},inplace=True)

# Map party abbreviations to fuller name
sov18_df['Party'] = sov18_df['Party'].map(party_map)

# Write common columns to disk
sov18_df.to_csv(
    boco_cleaned_path + '2018-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov18_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2163307100,1377,1149,Representative to the United States Congress -...,Joe Neguse,Democratic,723
1,2163307100,1377,1149,Representative to the United States Congress -...,Peter Yu,Republican,337


### 2017

In [12]:
sov17_df = pd.read_excel(
    boco_original_path + '2017-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov17_df = sov17_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

# Subset to common columns
sov17_df = sov17_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov17_df.replace({'Choice Name':choice_name_map},inplace=True)

# Map party abbreviations to fuller name
sov17_df['Party'] = sov17_df['Party'].map(party_map)

# Write common columns to disk
sov17_df.to_csv(
    boco_cleaned_path + '2017-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov17_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2181007800,427,230,City of Boulder Council Candidates,Bill Rigler,NaN,73
1,2181007801,640,209,City of Boulder Council Candidates,Bill Rigler,NaN,43


### 2016

In [13]:
sov16_df = pd.read_excel(
    boco_original_path + '2016-general-sov.xls',
    dtype={'Precinct Name':str}
)

sov16_df = sov16_df.rename(
    columns = {
        'Total\nBallots':'Total Ballots',
        'Total\nVotes':'Total Votes',
        'Contest Title':'Contest Name',
        'Active\nVoters':'Active Voters',
        'Total\nUnder\nVotes':'Total Under Votes',
        'Total\nOver\nVotes':'Total Over Votes'
    }
)

These mappings must be made manually.

In [14]:
sov16_df['Party'] = np.nan

sov16_choice_map = {
    # President
    'Donald J. Trump / Michael R. Pence':'Republican',
    'Hillary Clinton / Tim Kaine':'Democratic',
    'Jill Stein / Ajamu Baraka':'Green',
    'Gary Johnson / Bill Weld':'Libertarian',
    'Evan McMullin / Nathan Johnson':'Independent',
    'Ryan Alan Scott / Bruce Kendall Barnard':'Independent',
    'Darrell L. Castle / Scott N. Bradley':'Constitution',
    'Mike Smith / Daniel White':'Independent',
    'Don Willoughby - Unaffiliated':'Unaffiliated',
    'Alyson Kennedy / Osborne Hart  Socialist Workers':'Socialist Workers',
    'Bradford Lyttle / Hannah Walsh  Nonviolent Resistance/Pacifist':'Nonviolent Resistance/Pacifist',
    'Brian Anthony Perry / Michael Byron Nelson - Unaffiliated':'Unaffiliated',
    'Bruce Lohmiller / J.R. Smith - Green / Democratic':'Green / Democratic',
    'Cherunda Fox / Roger Kushner - Unaffiliated':'Unaffiliated',
    'Chris Keniston / Deacon Taylor  Veterans of America':'Veterans of America',
    'Corey Sterner / Jeff Ryan - Unaffiliated / Republican':'Unaffiliated / Republican',
    'David Perry / Eric "Rick" Seiley - Republican':'Republican',
    'Emidio Soltysik / Angela Nicole Walker  Socialist USA':'Socialist USA',
    'Frank Atwood / Blake Huber  Approval Voting':'Approval Voting',
    'Gloria Estela La Riva / Dennis J. Banks  Socialism and Liberation':'Socialism and Liberation',
    'James Hedges / Bill Bayes  Prohibition':'Prohibition',
    'Joseph Allen Maldonado / Douglas K. Terranova  Independent People':'Independent People',
    'Kyle Kenley Kopitke / Nathan R. Sorenson  Independent American':'Independent American',
    'Laurence Kotlikoff / Edward Leamer  Kotlikoff for President':'Kotlikoff for President',
    'Michael A. Maturen / Juan Munoz  American Solidarity':'American Solidarity',
    'Rocky Roque De La Fuente / Michael Steinberg  American Delta':'American Delta',
    'Rod Silva / Richard C. Silva  Nutrition':'Nutrition',
    'Thomas J. Nieman / Bernie Jackson - Unaffiliated / Libertarian':'Unaffiliated / Libertarian',
    'Tom Hoefling / Steve Schulin  America\'s':'America\'s',
    
    # Senate
    'Arn Menconi':'Green',
    'Bill Hammons':'Unity',
    'Michael Bennet':'Democratic',
    'Dan Chapin':'Unaffiliated',
    'Darryl Glenn':'Republican',
    'Don Willoughby - Unaffiliated':'Unaffiliated',
    'Lily Tang Williams':'Libertarian',
    'Paul Noel Fiorino':'Unaffiliated',
    
    # CO-2
    'Jared Polis':'Democratic',
    'Nicholas Morse':'Republican',
    'Richard Longstreth':'Libertarian',
    
    # CO-4
    'Bob Seay':'Democratic',
    'Bruce Griffith':'Libertarian',
    'Donald Howbert - Republican':'Republican',
    'Ken Buck':'Republican',
    
    # Others
    'Alice Madden':'Democratic',
    'Heidi Ganahl':'Republican',
    'Bob Owens':'Democratic',
    'Suzanne M. Sharkey':'Republican',
    'Edie Hooton':'Democratic',
    'Corey Piper':'Republican',
    'Jonathan Singer':'Democratic',
    'Bob Dillon':'Republican',
    'Mike Foote':'Democratic',
    'KC Becker':'Democratic',
    'Karen Nelson':'Republican',
    'Kim Tavendale':'Libertarian',
    'Matt Gray':'Democratic',
    'M. Peter Spraitz':'Republican',
    'Stan Garnett':'Democratic',
    'Kevin Sipple':'Republican',
    'Elise Jones':'Democratic',
    'Deb Gardner':'Democratic',
    'Paul Danish':'Republican',
}

# Map party abbreviations to fuller name
sov16_df['Party'] = sov16_df['Choice Name'].map(sov16_choice_map)

# Cast choices to consistent names
sov16_df.replace({'Choice Name':choice_name_map},inplace=True)

# Remove parties from candidate names
for _candidate,_party in sov16_choice_map.items():
    _ix = sov16_df[sov16_df['Choice Name'] == _candidate].index
    sov16_df.loc[_ix,'Choice Name'] = sov16_df.loc[_ix,'Choice Name'].str.replace(_party,'',regex=False)
    sov16_df.loc[_ix,'Choice Name'] = sov16_df.loc[_ix,'Choice Name'].str.replace('-','',regex=False)
    sov16_df.loc[_ix,'Choice Name'] = sov16_df.loc[_ix,'Choice Name'].str.strip()

In [15]:
# Subset to common columns
sov16_df = sov16_df.reindex(columns=common_cols)

# Write common columns to disk
sov16_df.to_csv(
    boco_cleaned_path + '2016-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov16_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2163307100,1291.0,1174,Presidential Electors,Hillary Clinton / Tim Kaine,Democratic,724
1,2163307100,1291.0,1174,Presidential Electors,Donald J. Trump / Michael R. Pence,Republican,327


### 2015

In [16]:
sov15_df = pd.read_excel(
    boco_original_path + '2015-general-sov.xls',
    dtype={'Precinct Name':str}
)

# Replace bizarre dual-precinct precinct
sov15_df = sov15_df.replace({'Precinct Name':{'2181007800, 2181207403':'2181007800'}})

# Rename columns
sov15_df = sov15_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

# Subset to common columns
sov15_df = sov15_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov15_df.replace({'Choice Name':choice_name_map},inplace=True)

# Map party abbreviations to fuller name
# sov15_df['Party'] = sov15_df['Party'].map(party_map)

# Write common columns to disk
sov15_df.to_csv(
    boco_cleaned_path + '2015-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov15_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2181007800,417,223,City of Boulder Council Candidates,Jared Kaszuba,NaN,16
1,2181007800,417,223,City of Boulder Council Candidates,Lisa Morzel,NaN,59


In [17]:
sov15_df['Party'].value_counts()

Series([], Name: Party, dtype: int64)

### 2014

In [18]:
sov14_df = pd.read_excel(
    boco_original_path + '2014-general-sov.xls',
    dtype={'Precinct \nName':str}
)

sov14_df = sov14_df.rename(
    columns = {
        'Precinct \nName':'Precinct Name',
        'Total \nBallots':'Total Ballots',
        'Total \nVotes':'Total Votes',
        'Contest Title':'Contest Name'
    }
)

# Subset to common columns
sov14_df = sov14_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov14_df.replace({'Choice Name':choice_name_map},inplace=True)

# Map party abbreviations to fuller name
sov14_df['Party'] = sov14_df['Party'].map(party_map)

# Write common columns to disk
sov14_df.to_csv(
    boco_cleaned_path + '2014-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov14_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2163307100,1118,827,United States Senator,Mark Udall,Democratic,467
1,2163307100,1118,827,United States Senator,Cory Gardner,Republican,314


### 2013

In [19]:
sov13_df = pd.read_excel(
    boco_original_path + '2013-general-sov.xls',
    sheet_name='SOV',
    dtype={'Precinct Name':str}
)

sov13_df = sov13_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov13_df['Contest Name'] = sov13_df['Contest Name'].str.title()

# Subset to common columns
sov13_df = sov13_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov13_df.replace({'Choice Name':choice_name_map},inplace=True)

# Write common columns to disk
sov13_df.to_csv(
    boco_cleaned_path + '2013-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov13_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2181007800,426,249,City Of Boulder Council Candidates,Mary Young,NaN,66
1,2181007800,426,249,City Of Boulder Council Candidates,John Gerstle,NaN,58


### 2012
2012 takes a **lot** of effort to clean up.

In [20]:
sov12_raw_df = pd.read_excel(
    boco_original_path + '2012-general-sov.xls',
    header=3
)

c0 = sov12_raw_df.isnull().sum(1) == 16
c1 = sov12_raw_df.iloc[:,0] != 'Continued'

sov12_title_indices = sov12_raw_df[c0 & c1].index

Presidental electors are spread across three panels, so this needs special care.

In [21]:
# First panel of presidental electors
panel_df0 = sov12_raw_df.iloc[4:236].replace({'%':np.nan}).dropna(how='all',axis=1)
panel_df0.columns = sov12_raw_df.loc[2,panel_df0.columns].str.replace('\r\n',' ').str.strip()

# Second panel of presidential electors, ignoring redundant columns
panel_df1 = sov12_raw_df.iloc[243:475].replace({'%':np.nan}).dropna(how='all',axis=1)
panel_df1.columns = sov12_raw_df.loc[241,panel_df1.columns].str.replace('\r\n',' ').str.strip()
cleaned_panel_df1 = panel_df1.iloc[:,[0,7,8,9,10,11,12,13]]

# Third panel of presidential electors, ignoring redundant columns
panel_df2 = sov12_raw_df.iloc[482:714].replace({'%':np.nan}).dropna(how='all',axis=1)
panel_df2.columns = sov12_raw_df.loc[480,panel_df2.columns].str.replace('\r\n',' ').str.strip()
cleaned_panel_df2 = panel_df2.iloc[:,[0,7,8,9]]

# Join first and second panels together on Precinct
sov12_presidential = pd.merge(
    left = panel_df0,
    right = cleaned_panel_df1,
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# Join joined panels with third panel on Precinct
sov12_presidential = pd.merge(
    left = sov12_presidential,
    right = cleaned_panel_df2,
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# Melt wide data to tidy data
sov12_pres_melted = pd.melt(
    frame = sov12_presidential,
    id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
    value_vars = sov12_presidential.columns[7:],
    var_name = 'Choice Name',
    value_name = 'Total Votes'
)

# Add Contest Name
sov12_pres_melted['Contest Name'] = 'Presidential Electors'

# sov12_pres_melted.head()

Extract the data for the other contest panels.

In [22]:
# Dictionary to store results for later concatenation
cleaned_panels = {}

# Define the panel ranges for other contests
panel_ranges = sov12_raw_df.iloc[sov12_title_indices,0].to_dict()

# Loop through other panels
for i,(k,v) in enumerate(panel_ranges.items()):
    
    # Ignore the first three panels corresponding to presidential electors
    if i > 2:
        
        # Name of contest is first row of panel
        contest_name = sov12_raw_df.iloc[k,0]
        
        # Row corresponding to panel names
        columns = k+2
        
        # The data starts at this row
        data_start = k+4
        
        # If we're not in the last panel, the data stops two rows before the next panel
        if i + 1 < len(panel_ranges):
            data_stop = list(panel_ranges.keys())[i+1] - 2
        else:
            data_stop = sov12_raw_df.index.max()
        
        # Slice the rows with data, drop blank columns
        panel_df = sov12_raw_df.iloc[data_start:data_stop].replace({'%':np.nan}).dropna(how='all',axis=1)
        
        # Rename the columns based on the column row
        panel_df.columns = sov12_raw_df.loc[columns,panel_df.columns].str.replace('\r\n',' ').str.strip()
        
        # Melt the wide data down to tidy data
        melted_df = pd.melt(
            frame = panel_df,
            id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
            value_vars = panel_df.columns[7:],
            var_name = 'Choice Name',
            value_name = 'Total Votes'
        )

        # Store in the dictionary
        cleaned_panels[contest_name.title()] = melted_df

Concatenate the data together.

In [23]:
# Concatenate the cleaned panels together
sov12_cleaned_df = pd.concat(cleaned_panels).reset_index(0)

# Ignore rows with totals
sov12_cleaned_df = sov12_cleaned_df[sov12_cleaned_df['Choice Name'] != 'Totals']

# Rename columns
sov12_cleaned_df.rename(columns={'level_0':'Contest Name'},inplace=True)

# Add in the presidential elector data
sov12_cleaned_df = pd.concat([sov12_pres_melted,sov12_cleaned_df])

# Reset the index
sov12_cleaned_df.reset_index(drop=True,inplace=True)

In [24]:
# Find the Choice Names containing 'Democrat' or 'Republican'
dem_contests = sov12_cleaned_df.groupby('Contest Name').agg({'Choice Name':'unique'})['Choice Name'].apply(lambda x:bool([i for i in x if 'Democrat' in i]))
rep_contests = sov12_cleaned_df.groupby('Contest Name').agg({'Choice Name':'unique'})['Choice Name'].apply(lambda x:bool([i for i in x if 'Republican' in i]))

# Find Contests with either a Democrat or Republican running
both_contests = dem_contests | rep_contests
partisan_contests = both_contests[both_contests]

# Subset to the partisan contests and split Choice from Party on 4 spaces
choice_party = sov12_cleaned_df.loc[sov12_cleaned_df['Contest Name'].isin(partisan_contests.index),'Choice Name'].str.split('    ')
choice_s = choice_party.apply(lambda x:x[0])
party_s = choice_party.apply(lambda x:x[1])

# Create a new column for Party
sov12_cleaned_df['Party'] = np.nan

# Relabel the Choice Name on the candidate before the four spaces
sov12_cleaned_df.loc[choice_s.index,'Choice Name'] = choice_s.values

# Add the Party from after the four spaces
sov12_cleaned_df.loc[party_s.index,'Party'] = party_s.values

Add complete precinct values.

In [25]:
sov12_cleaned_df['Precinct'] = sov12_cleaned_df['Precinct'].astype(str)
sov12_cleaned_df.rename(columns = {'Precinct':'Precinct Short'},inplace=True)

sov12_cleaned_df = pd.merge(
    left = sov12_cleaned_df,
    right = sov22_raw_df[['Precinct Name (Short)','Precinct Name']].drop_duplicates(),
    left_on = 'Precinct Short',
    right_on = 'Precinct Name (Short)',
    how = 'left'
)

sov12_cleaned_df = sov12_cleaned_df[[c for c in sov12_cleaned_df.columns if 'Short' not in c]]

# simple_precincts = ~(sov12_cleaned_df['Precinct'].str.contains(',') | sov12_cleaned_df['Precinct'].str.contains('Landowner'))

# sov12_cleaned_df.loc[simple_precincts.index,'Precinct'] = '2163307' + sov12_cleaned_df.loc[simple_precincts.index,'Precinct']

Rename columns to align with others.

In [26]:
sov12_cleaned_df.rename(
    columns = 
    {
        'Precinct':'Precinct Name',
        'Total Ballots Cast':'Total Ballots',
        'Registered Voters':'Active Voters'
    },
    inplace=True
)

# Subset to common columns
sov12_df = sov12_cleaned_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov12_df.replace({'Choice Name':choice_name_map},inplace=True)

# Write common columns to disk
sov12_df.to_csv(
    boco_cleaned_path + '2012-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov12_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2181207100,1369,1032,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,American,0
1,2181207101,1369,1040,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,American,2


### 2011

In [27]:
sov11_raw_df = pd.read_excel(
    boco_original_path + '2011-general-sov.xls',
)

In [28]:
# First panel of Boulder city council candidates
sov11_panel1_df = sov11_raw_df.loc[11:101]
sov11_panel1_df.columns = sov11_raw_df.loc[10]

# Second panel of Boulder city council candidates
sov11_panel2_df = sov11_raw_df.loc[108:199]
sov11_panel2_df.columns = sov11_raw_df.loc[107]
sov11_panel2_df = sov11_panel2_df.iloc[:,[0,7,8,9,10,11,12,13]]

# Join first and second panels together on Precinct
sov11_bcc = pd.merge(
    left = sov11_panel1_df,
    right = sov11_panel2_df,
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# Melt wide data to tidy data
sov11_bcc_melted = pd.melt(
    frame = sov11_bcc,
    id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
    value_vars = sov11_bcc.columns[8:],
    var_name = 'Choice Name',
    value_name = 'Total Votes'
)

# Add Contest Name
sov11_bcc_melted['Contest Name'] = 'City of Boulder Council Candidates'

In [29]:
# First panel of Layfayette city council candidates
sov11_panel3_df = sov11_raw_df.loc[303:321]
sov11_panel3_df.columns = sov11_raw_df.loc[302]

# # Melt wide data to tidy data
sov11_lcc_melted = pd.melt(
    frame = sov11_panel3_df,
    id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
    value_vars = sov11_panel3_df.columns[8:],
    var_name = 'Choice Name',
    value_name = 'Total Votes'
)

# # Add Contest Name
sov11_lcc_melted['Contest Name'] = 'City of Lafayette Council Candidates'

In [30]:
sov11_starts = sov11_raw_df.loc[sov11_raw_df.iloc[:,0] == 'Precinct']
sov11_stops = sov11_raw_df.loc[sov11_raw_df.iloc[:,0] == 'Totals']

In [31]:
# Dictionary to store results for later concatenation
cleaned_panels = {}

# Loop through other panels
for i,(start,stop) in enumerate(zip(sov11_starts.index, sov11_stops.index)):
    
    # Ignore the first four panels corresponding to Boulder and Lafayette City Council contests
    if i > 4:
        
        # Spacing for name of contest changes after 44th race
        if i < 44:
            contest_name = sov11_raw_df.iloc[start-3,6]
        
        else:
            contest_name = sov11_raw_df.iloc[start-2,6]
        
        # Row corresponding to column names
        columns = start

        # The data starts at this row
        data_start = start + 1

        # If we're not in the last panel, the data stops two rows before the next panel
        data_stop = stop - 1

        # Slice the rows with data, drop blank columns
        panel_df = sov11_raw_df.iloc[data_start:data_stop]#.dropna(how='all',axis=1)

        # Rename the columns based on the column row
        panel_df.columns = sov11_raw_df.loc[columns,panel_df.columns].str.replace('\r\n',' ').str.strip()

        # Melt the wide data down to tidy data
        melted_df = pd.melt(
            frame = panel_df,
            id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
            value_vars = panel_df.columns[7:],
            var_name = 'Choice Name',
            value_name = 'Total Votes'
        )
        
        melted_df = melted_df[melted_df['Choice Name'] != "Totals"]

        # Store in the dictionary
        cleaned_panels[contest_name.title()] = melted_df

In [32]:
sov11_concat_df = pd.concat(cleaned_panels)
sov11_concat_df.dropna(subset=['Choice Name','Total Votes'],inplace=True)
sov11_concat_df.reset_index(0,inplace=True)
sov11_concat_df.rename(columns={'level_0':'Contest Name'},inplace=True)

sov11_concat_df = pd.concat([sov11_bcc_melted,sov11_lcc_melted,sov11_concat_df])
sov11_concat_df.reset_index(drop=True,inplace=True)

In [33]:
# TODO: Align 2011 precincts with 2022 precinct names

In [34]:
# Rename columns
sov11_concat_df.rename(
    columns = 
    {
        'Precinct':'Precinct Name',
        'Total Ballots Cast':'Total Ballots',
        'Registered Voters':'Active Voters'
    },
    inplace=True
)

# Subset to common columns
sov11_df = sov11_concat_df.reindex(columns=common_cols)

# Cast choices to consistent names
sov11_df.replace({'Choice Name':choice_name_map},inplace=True)

# Write common columns to disk
sov11_df.to_csv(
    boco_cleaned_path + '2011-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov11_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,073,289,157,City of Boulder Council Candidates,Mark Gelband,NaN,27
1,076,330,200,City of Boulder Council Candidates,Mark Gelband,NaN,32


### 2010

In [35]:
# Read in data
sov10_raw_df = pd.read_excel(
    boco_original_path + '2010-general-sov.xls',
)

# Drop all the empty rows and columns
sov10_raw_df.dropna(axis=0,how='all',inplace=True)
sov10_raw_df.dropna(axis=1,how='all',inplace=True)

# Drop mostly empty rows
bad_rows_0 = sov10_raw_df[sov10_raw_df.iloc[:,6] == 'Continued'].index.tolist()
sov10_raw_df.drop(bad_rows_0 + [0],inplace=True)

# Reset index
sov10_raw_df.reset_index(drop=True,inplace=True)

In [36]:
sov10_starts = sov10_raw_df.loc[sov10_raw_df.iloc[:,0] == 'Precinct']
sov10_stops = sov10_raw_df.loc[sov10_raw_df.iloc[:,0] == 'Canvass Report  —  Total Voters  —  Official ']
print(sov10_starts.shape, sov10_starts.shape)

(302, 22) (302, 22)


Extract panels.

In [37]:
# Container for storing sub-panels
sov10_contests_d = {}

# Container for bad panels
bad_starts = []

# For each panel
for (_start,_stop) in zip(sov10_starts.index,sov10_stops.index):
    try:
        # Contest name is row before the panel
        _contest = sov10_raw_df.iloc[_start-1,0].title()
        
        # Clean up column names
        _cols = sov10_raw_df.iloc[_start].dropna()
        _cols = [_c.replace('\n',' ') for _c in _cols if _c != 'Continued']

        # Extract the panel
        _df = sov10_raw_df.copy().iloc[_start+1:_stop-1,:]
        _df.dropna(axis=1,inplace=True,how='all')

        # Rename the columns
        _df.columns = _cols

        # Add the panel to the container
        if _contest in sov10_contests_d:
            sov10_contests_d[_contest].append(_df)
        else:
            sov10_contests_d[_contest] = [_df]
            
    # Spacing on some panels is off
    except:
        
        try:
            # Contest name is row before the panel
            _contest = sov10_raw_df.iloc[_start-1,0].title()

            # Clean up column names
            _cols = sov10_raw_df.iloc[_start].dropna()
            _cols = [_c.replace('\n',' ') for _c in _cols if _c != 'Continued']

            # Extract the panel
            # Off-by-one error in the totals row, omit since it's dropped
            _df = sov10_raw_df.copy().iloc[_start+1:_stop-2,:]
            _df.dropna(axis=1,inplace=True,how='all')

            # Rename the columns
            _df.columns = _cols

            # Add the panel to the container
            if _contest in sov10_contests_d:
                sov10_contests_d[_contest].append(_df)
            else:
                sov10_contests_d[_contest] = [_df]
                
        except:
            bad_starts.append(_start)
            pass
    
print(len(bad_starts))

bad_starts

0


[]

Concatenate the multipanel contests together manually.

In [38]:
# 'United States Senator'
sov10_senator = pd.merge(
    left = pd.concat(sov10_contests_d['United States Senator'][:7]),
    right = pd.concat(sov10_contests_d['United States Senator'][7:]).iloc[:,[0,7,8,9]],
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# Governor/Lieutenant Governor
sov10_governor = pd.merge(
    left = pd.concat(sov10_contests_d['Governor/Lieutenant Governor'][:7]),
    right = pd.concat(sov10_contests_d['Governor/Lieutenant Governor'][7:]).iloc[:,[0,7,8,9]],
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

Concatenate the other contests together.

In [39]:
sov10_contests_concat_d = {
    'United States Senator':sov10_senator,
    'Governor/Lieutenant Governor':sov10_governor
}

for _contest,_results_l in sov10_contests_d.items():
    if _contest not in sov10_contests_concat_d.keys():
        sov10_contests_concat_d[_contest] = pd.concat(_results_l)

Melt the data down into a tidy format.

In [40]:
_keep_cols = ['Precinct','Total Ballots Cast','Registered Voters']

melted_l = []

for _contest,_df in sov10_contests_concat_d.items():
    _cand_cols = _df.columns[7:].tolist()
    _tomelt_df = _df.loc[:,_keep_cols + _cand_cols]

    _melted_df = pd.melt(
        _tomelt_df,
        id_vars = _keep_cols,
        value_vars = _cand_cols,
        var_name = 'Choice Name',
        value_name = 'Total Votes'
    )

    _melted_df['Contest Name'] = _contest
    
    melted_l.append(_melted_df)

Concatenate the melted data, clean up data, write to disk.

In [41]:
sov10_df = pd.concat(melted_l,ignore_index=True)

# Remove totals
sov10_df = sov10_df[sov10_df['Choice Name'] != 'Totals']

# Remove white space
sov10_df['Choice Name'] = sov10_df['Choice Name'].str.strip()

# Cast choices to consistent names
sov10_df.replace({'Choice Name':choice_name_map},inplace=True)
sov10_df.replace(
    {'Choice Name':
     {
         'Willie Travis Chambers / Ed E. Coron - DEM / REP':'Willie Travis Chambers / Ed E. Coron',
         'Peter J. Carr / Anoinette M. Schaeffer - Not':'Peter J. Carr / Anoinette M. Schaeffer'
     }
    },
    inplace=True
)

# Extract party
sov10_df['Party'] = sov10_df['Choice Name'].str.extract(r'\b([A-Z]{3})\b')

# Map party abbreviations to fuller name
sov10_df['Party'] = sov10_df['Party'].map(party_map)

# Extract candidate name(s)
sov10_df['Choice Name'] = sov10_df['Choice Name'].str.replace(r'\b([A-Z]{3})\b','',regex=True)
sov10_df['Choice Name'] = sov10_df['Choice Name'].str.replace('-','',regex=False)
sov10_df['Choice Name'] = sov10_df['Choice Name'].str.strip()

# Rename columns
sov10_df.rename(
    columns = {
        'Precinct':'Precinct Name',
        'Total Ballots Cast':'Total Ballots',
        'Registered Voters':'Active Voters',
    },
    inplace=True
)

# Subset to common columns
sov10_df = sov10_df.reindex(columns=common_cols)

# Write common columns to disk
sov10_df.to_csv(
    boco_cleaned_path + '2010-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov10_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,001,373,268,United States Senator,Ken Buck,Republican,111
1,002,659,422,United States Senator,Ken Buck,Republican,159


### 2009

### 2008

In [42]:
# Read in data
sov08_raw_df = pd.read_excel(
    boco_original_path + '2008-general-sov.xls',
)

# Drop all the empty rows and columns
sov08_raw_df.dropna(axis=0,how='all',inplace=True)
sov08_raw_df.dropna(axis=1,how='all',inplace=True)

# Drop mostly empty rows
# sov08_bad_rows_0 = sov08_raw_df[pd.notnull(sov08_raw_df.iloc[:,5])].index.tolist()
sov08_bad_rows_1 = sov08_raw_df[sov08_raw_df.iloc[:,6] == 'Continued'].index.tolist()
# sov08_raw_df.drop(sov08_bad_rows_0 + sov08_bad_rows_1 + [0,3],inplace=True)
sov08_raw_df.drop([0,3]+sov08_bad_rows_1,inplace=True)

# Reset index
sov08_raw_df.reset_index(drop=True,inplace=True)

In [43]:
sov08_starts = sov08_raw_df.loc[sov08_raw_df.iloc[:,0] == 'Precinct']
sov08_stops = sov08_raw_df.loc[sov08_raw_df.iloc[:,0] == 'Canvass Report  —  Total Voters  —  Official ']
print(sov08_starts.shape, sov08_starts.shape)

(381, 26) (381, 26)


In [44]:
# Container for storing sub-panels
sov08_contests_d = {}

# Container for bad panels
sov08_bad_starts = []

# For each panel
for (_start,_stop) in zip(sov08_starts.index,sov08_stops.index):
#     try:
    # Contest name is row before the panel
    _contest = sov08_raw_df.iloc[_start-1,0].title()

    # Clean up column names
    _cols = sov08_raw_df.iloc[_start].dropna()
    _cols = [_c.replace('\n',' ') for _c in _cols if _c != 'Continued']

    # Extract the panel
    _df = sov08_raw_df.copy().iloc[_start+1:_stop-1,:]
    _df.dropna(axis=1,inplace=True,how='all')

    # Rename the columns
    _df.columns = _cols

    # Add the panel to the container
    if _contest in sov08_contests_d:
        sov08_contests_d[_contest].append(_df)
    else:
        sov08_contests_d[_contest] = [_df]
    
#     except:
#         sov08_bad_starts.append((_start,_stop))
#         pass
    
print(len(sov08_bad_starts))

0


In [45]:
# United States President
sov08_president = pd.merge(
    left = pd.concat(sov08_contests_d['Presidential Electors'][:7]),
    right = pd.concat(sov08_contests_d['Presidential Electors'][7:14]).iloc[:,[0,7,8,9,10,11,12,13]],
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

sov08_president = pd.merge(
    left = sov08_president,
    right = pd.concat(sov08_contests_d['Presidential Electors'][14:]).iloc[:,[0,7,8]],
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# United States Senator
sov08_senator = pd.merge(
    left = pd.concat(sov08_contests_d['United States Senator'][:7]),
    right = pd.concat(sov08_contests_d['United States Senator'][7:]).iloc[:,[0,7]],
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

In [46]:
sov08_contests_concat_d = {
    'President':sov08_president,
    'United States Senator':sov08_senator
}

for _contest,_results_l in sov08_contests_d.items():
    if _contest not in sov08_contests_concat_d.keys():
        sov08_contests_concat_d[_contest] = pd.concat(_results_l)

In [47]:
_keep_cols = ['Precinct','Total Ballots Cast','Registered Voters']

melted_l = []

for _contest,_df in sov08_contests_concat_d.items():
    _cand_cols = _df.columns[7:].tolist()
    _tomelt_df = _df.loc[:,_keep_cols + _cand_cols]

    _melted_df = pd.melt(
        _tomelt_df,
        id_vars = _keep_cols,
        value_vars = _cand_cols,
        var_name = 'Choice Name',
        value_name = 'Total Votes'
    )

    _melted_df['Contest Name'] = _contest
    
    melted_l.append(_melted_df)

In [48]:
sov08_df = pd.concat(melted_l,ignore_index=True)

# Remove totals
sov08_df = sov08_df[~sov08_df['Choice Name'].str.contains('Totals')]

# Remove white space
sov08_df['Choice Name'] = sov08_df['Choice Name'].str.strip()

# Cast choices to consistent names
sov08_df.replace({'Choice Name':choice_name_map},inplace=True)

# Errant commas
sov08_df['Total Ballots Cast'] = sov08_df['Total Ballots Cast'].str.replace(',','').fillna(0).astype(int)
sov08_df['Total Votes'] = sov08_df['Total Votes'].fillna(0).astype(int)

In [49]:
# Extract parties
sov08_parties = [
    'Democratic','Objectivist','Unaffiliated','Socialist, USA','U.S. Pacifist','Socialism and','Republican',
    'Boston Tea','Socialist Workers','Prohibition','HeartQuake \'08','Green','Libertarian','Constitution',
    'America\'s','American Constitution','Unity'
]

# Transfer party names from Choice Name to Party
for party in sov08_parties:
    _ix = sov08_df['Choice Name'].str.contains(party)
    sov08_df.loc[_ix,'Party'] = party
    sov08_df.loc[_ix,'Choice Name'] = sov08_df.loc[_ix,'Choice Name'].str.replace(party,'',regex=False)



In [50]:
# Rename columns
sov08_df.rename(
    columns = {
        'Precinct':'Precinct Name',
        'Total Ballots Cast':'Total Ballots',
        'Registered Voters':'Active Voters',
    },
    inplace=True
)

# Subset to common columns
sov08_df = sov08_df.reindex(columns=common_cols)

# TODO: Need to diagnose where these missing values are coming from, but drop for now
sov08_df.dropna(subset=['Precinct Name','Active Voters'],inplace=True)

# Write common columns to disk
sov08_df.to_csv(
    boco_cleaned_path + '2008-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov08_df.head(2)

,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,001,426,323,President,John McCain / Sarah Palin,Republican,129
1,002,649,523,President,John McCain / Sarah Palin,Republican,181


### Combine together
Concatenate and clean up.

In [51]:
all_sov_df = pd.concat({
    2008:sov08_df,
    2010:sov10_df,
    2011:sov11_df,
    2012:sov12_df,
    2013:sov13_df,
    2014:sov14_df,
    2015:sov15_df,
    2016:sov16_df,
    2017:sov17_df,
    2018:sov18_df,
    2019:sov19_df,
    2020:sov20_df,
    2021:sov21_df,
    2022:sov22_df
})

# Clean up index
all_sov_df = all_sov_df.reset_index(0).reset_index(drop=True)

# Rename column
all_sov_df.rename(columns={'level_0':'Year'},inplace=True)

# Drop rows that have no ballots or voters
all_sov_df.dropna(subset=['Active Voters','Total Ballots'],inplace=True)

# Cast to proper types
all_sov_df['Active Voters'] = all_sov_df['Active Voters'].astype(int)
all_sov_df['Total Ballots'] = all_sov_df['Total Ballots'].astype(int)
all_sov_df['Total Votes'] = all_sov_df['Total Votes'].astype(int)
all_sov_df['Precinct Name'] = all_sov_df['Precinct Name'].astype(str)

# Number of rows
print("There are {0:,} rows.".format(len(all_sov_df)))

# Write to disk
all_sov_df.to_csv(
    boco_cleaned_path + 'all-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
all_sov_df.head()

There are 224,227 rows.


,Year,Precinct Name,Active Voters,Total Ballots,Contest Name,Choice Name,Party,Total Votes
0,2008,001,426,323,President,John McCain / Sarah Palin,Republican,129
1,2008,002,649,523,President,John McCain / Sarah Palin,Republican,181
2,2008,003,1235,1063,President,John McCain / Sarah Palin,Republican,325
3,2008,004,1132,948,President,John McCain / Sarah Palin,Republican,286
4,2008,005,402,342,President,John McCain / Sarah Palin,Republican,142


In [ ]:
all_sov_df.groupby(['Year','Party']).apply(len)

## Feature engineering

### Filter to City of Boulder (CoB) precincts

In [ ]:
# Precincts for 2021
precincts_21 = sov21_df.loc[sov21_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_21),2021))

# Precincts for 2019
precincts_19 = sov19_df.loc[sov19_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_19),2019))

# For 2017
precincts_17 = sov17_df.loc[sov17_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_17),2017))

# For 2015
precincts_15 = sov15_df.loc[sov15_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_15),2015))

# For 2013
precincts_13 = sov13_df.loc[sov13_df['Contest Name'] == 'City Of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_13),2013))

Identify the union of all precincts for CoB elections.

In [ ]:
precints_union = sorted(list(set(precincts_13) | set(precincts_15) | set(precincts_17) | set(precincts_19)))
print("There are {0:,} total precincts for City of Boulder Council candidates".format(len(precints_union)))
precints_union[:5]

Write out, because it's useful.

In [ ]:
with open('cob_precincts.json','w') as f:
    json.dump(precints_union,f)

### Contests

Pivot table to find contests that were city-wide.

In [ ]:
def citywide_pivot(sov_df,precincts_union,year):
    
    # Pivot precincts and contests to find contests with null values
    citywide_contests_pivot = pd.pivot_table(
        data = sov_df.loc[sov_df['Precinct Name'].isin(precints_union)],
        values = 'Total Votes',
        index = 'Precinct Name',
        columns = 'Contest Name',
    )
    
    # Drop columns (contests) with NaNs: not city-wide contests
    # citywide_contests = citywide_contests_pivot.dropna(axis=1,how='any').columns.tolist()
    null_precincts = citywide_contests_pivot.isnull().sum()
    citywide_contests = null_precincts[null_precincts <= 2].index.tolist()
    
    # Subset the statement of votes to City of Boulder precincts and contests
    c0 = sov_df['Precinct Name'].isin(precints_union)
    c1 = sov_df['Contest Name'].isin(citywide_contests)
    c2 = ~sov_df['Choice Name'].isin(["No/Against",'NO/AGAINST','No','Against the Measure','NO'])
    cob = sov_df[c0 & c1 & c2]

    # Pivot precincts by contest-choice with values as vote fraction
    cw_pivot = pd.pivot_table(
        data = cob,
        index = 'Precinct Name',
        columns = ['Contest Name','Choice Name'],
        values = ['Fraction'],
        fill_value = 0
    )
    
    # Add the year to columns to prevent duplicates, flatten multi-index column name
    cw_pivot = pd.concat([cw_pivot],keys=[year],names=['Year'],axis=1) # https://stackoverflow.com/a/42094658/1574687
    cw_pivot.columns = cw_pivot.columns.map(' '.join) # pandas multiindex flatten
    
    # Compute precinct-level turnout for each contest
    turnout = pd.pivot_table(
        data = cob,
        index = 'Precinct Name',
        columns = 'Contest Name',
        values = ['Total Ballots','Active Voters']
    )
    
    # Divide total ballots cast by number of active voters
    # turnout = turnout['Total Ballots'].div(turnout['Active Voters'])
    # turnout.columns = [year + ' turnout ' + c for c in turnout.columns]
    
    # Concatenate turnout to the contest-level voteshares
    # precinct_df = pd.concat([cw_pivot,turnout],axis=1).fillna(0)
    precinct_df = cw_pivot.fillna(0)
    
    return precinct_df

In [ ]:
cw_pivot_21 = citywide_pivot(sov21_df,precints_union,'2021')
cw_pivot_20 = citywide_pivot(sov20_df,precints_union,'2020')
cw_pivot_19 = citywide_pivot(sov19_df,precints_union,'2019')
cw_pivot_18 = citywide_pivot(sov18_df,precints_union,'2018')
cw_pivot_17 = citywide_pivot(sov17_df,precints_union,'2017')
cw_pivot_16 = citywide_pivot(sov16_df,precints_union,'2016')
cw_pivot_15 = citywide_pivot(sov15_df,precints_union,'2015')
cw_pivot_14 = citywide_pivot(sov14_df,precints_union,'2014')
cw_pivot_13 = citywide_pivot(sov13_df,precints_union,'2013')
cw_pivot_12 = citywide_pivot(sov12_cleaned_df,precints_union,'2012')

cw_pivot_21.shape, cw_pivot_20.shape, cw_pivot_19.shape, cw_pivot_18.shape, cw_pivot_17.shape, cw_pivot_16.shape, cw_pivot_15.shape, cw_pivot_14.shape, cw_pivot_13.shape, cw_pivot_12.shape

Horizontally concatenate.

In [ ]:
results_df = pd.concat(
    objs = [
        cw_pivot_21,cw_pivot_20,cw_pivot_19,cw_pivot_18,cw_pivot_17,
        cw_pivot_16,cw_pivot_15,cw_pivot_14,cw_pivot_13,cw_pivot_12
    ],
    axis=1
)

results_df.columns = results_df.columns.to_flat_index()

results_df.to_csv('voting_results_2012_2021.csv')

results_df.head()